In [12]:
import arcpy

In [13]:
# ----- variables section ----
home_folder = arcpy.mp.ArcGISProject("CURRENT").homeFolder

xsl_file  = rf"{home_folder}\ArcGIS2Inport.xsl"
#enter the gdb name here if using one, e.g. mystuff.gdb
gdb_name = "InPort"
#do not include .shp if using a shapefile
fc_name = "PolygonExample"
# before running this notebook, you set the required constants below
#
# defaultEffectiveDate is always required. Example 2018-06
#    - defaultEffectiveDate (effective dates for contacts - there is no where to enter this in metadata so it must be entered in the xslt file)
# one of these is always required
#    - parentCatalogItemId (use when creating a new inport record, leave blank '' when updating an inport record)
#    - catalogItemId (use when updating an inport record, leave bland '' when creating a new inport record)
# the following are only needed if you do not provide these contacts directly in your metadata
#    - defaultPointOfContactEmail (in ArcGIS Pro metadata, define this contact in Resource>Points of Contact)
#    - defaultDataStewardEmail (in ArcGIS Pro metadata, define this contact in Overview>Citation Contacts)
#    - defaultMetadataContactEmail (in ArcGIS Pro metadata, define this contact in Metadata>Contacts)
#    - defaultOrganizationName (in ArcGIS Pro metadata, define this contact in Resource>Distribution>Distributor )
parameters = {
    'defaultEffectiveDate':"2021-01",
    'parentCatalogItemId':"99999",
    'catalogItemId':"",
    'defaultPointOfContactEmail':"pointofcontact@noaa.gov",
    'defaultDataStewardEmail':"datasteward@noaa.gov",
    'defaultMetadataContactEmail':"metadata@noaa.gov",
    'defaultOrganizationName':"Default OrgName"}
# ----- end variables section ------

In [14]:
# Read in the xsl file and replace the parameter values
with open(xsl_file, 'r') as file :
  file_data = file.read()

for k in parameters:
    print("'"+k+"'")
    # Replace the target string
    #file_data = file_data.replace("["+k+"]", parameters[k])
    file_data = file_data.replace("'"+k+"'", parameters[k])

# Write the file out again to the workspace folder
with open(xsl_file, 'w') as file:
  file.write(file_data)

'defaultEffectiveDate'
'parentCatalogItemId'
'catalogItemId'
'defaultPointOfContactEmail'
'defaultDataStewardEmail'
'defaultMetadataContactEmail'
'defaultOrganizationName'


In [15]:
import os
from arcpy import metadata as md

arcpy.env.overwriteOutput = True

if gdb_name:
    arcpy.env.workspace = rf"{home_folder}\{gdb_name}.gdb"
else:
    arcpy.env.workspace = rf"{home_folder}"
    
fcs = arcpy.ListFeatureClasses(f"{fc_name}*")

if len(fcs) == 0:
    print("Can't find feature class or shapefile")
else:
    print("Working")
    for fc in fcs:
        print(f"\nDataset: {os.path.basename(fc)}")
        fc_md = md.Metadata(fc)
        fc_md.synchronize("ALWAYS")
        fc_md.save()
        fc_md.exportMetadata(f"{home_folder}\{fc_name}_InPort.xml", "CUSTOM", "EXACT_COPY", xsl_file)
        
        del fc_md
        del fc
    print("Work Complete")
del fcs
    

Working

Dataset: PolygonExample

Dataset: PolygonExample_Polygon2PointsScript
Work Complete
